In [5]:

!pip install pyspark
!pip install pymongo
!pip install certifi

In [7]:

!hostname -I

#gives ip address
!curl ipecho.net/plain

#Gives ip addresses with port numbers
!sudo lsof -i -P -n | grep LISTEN

172.28.0.12 
34.16.144.90/bin/bash: line 1:  grep LISTEN: command not found
lsof: illegal option character: �
lsof: illegal option character: �
lsof 4.93.2
 latest revision: https://github.com/lsof-org/lsof
 latest FAQ: https://github.com/lsof-org/lsof/blob/master/00FAQ
 latest (non-formatted) man page: https://github.com/lsof-org/lsof/blob/master/Lsof.8
 usage: [-?abhKlnNoOPRtUvVX] [+|-c c] [+|-d s] [+D D] [+|-E] [+|-e s] [+|-f[gG]]
 [-F [f]] [-g [s]] [-i [i]] [+|-L [l]] [+m [m]] [+|-M] [-o [o]] [-p s]
 [+|-r [t]] [-s [p:s]] [-S [t]] [-T [t]] [-u s] [+|-w] [-x [fl]] [--] [names]
Use the ``-h'' option to get more help information.


In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split
from pyspark.sql.types import StringType, StructField, StructType
from pymongo import MongoClient
from pymongo.errors import ServerSelectionTimeoutError
import certifi
from bs4 import BeautifulSoup

# SparkSession oluşturma
spark = SparkSession.builder \
    .appName("TextProcessingFromMongoDB") \
    .getOrCreate()

# Kafka bağlantı ayarları
bootstrap_servers = "kendi_ayarlarınız"
security_protocol = "kendi_ayarlarınız"
sasl_mechanisms = "kendi_ayarlarınız"
sasl_username = "kendi_ayarlarınız"
sasl_password = "kendi_ayarlarınız"

# MongoDB bağlantısı kurma
try:
    client = MongoClient(
        "mongodb+srv://mehmet34:<pasword>@atlascluster.j3z8vqq.mongodb.net/",
        tlsCAFile=certifi.where()
    )
    db = client["mydatabase"]
    collection = db["mycollection1"]

    # İşlenmemiş verileri al
    documents = collection.find({"data": {"$exists": True}, "state": 0})

    # İşlenmemiş veri yoksa mesaj ver
    if collection.count_documents({"data": {"$exists": True}, "state": 0}) == 0:
        print("İşlenmemiş veri yok.")
    else:
        for document in documents:
            text = document['data']
            soup = BeautifulSoup(text, 'html.parser')

            # Gereksiz etiketleri temizle
            for a_tag in soup.find_all("a"):
                a_tag.decompose()
            for img_tag in soup.find_all("img"):
                img_tag.decompose()
            for s_tag in soup.find_all("strong"):
                s_tag.decompose()

            # Article etiketlerini bul ve işle
            articles = soup.find_all("article")
            print(document['url'])
            print("\n")

            text_sum_list = []
            for article in articles:
                paragraphs = article.find_all(["p", "h1", "h2"])
                combined_text = "\n".join([element.get_text() for element in paragraphs])
                text_sum = combined_text.strip().split()
                text_sum_list.extend(text_sum)

            query = {"_id": document["_id"]}
            new_data = {"$set": {"data": text_sum_list, "state": 1}}

            try:
                collection.update_one(query, new_data)
                print("Veri başarıyla liste olarak güncellendi")
            except Exception as e:
                print(e)

            print("#########")

    # Veriyi çekme
    data1 = list(collection.find({"state": 1}, {"url": 1, "data": 1, "_id": 0}))

    # Veri yapısını tanımlama
    schema = StructType([
        StructField("url", StringType(), True),
        StructField("data", StringType(), True)
    ])

    # RDD ve DataFrame oluşturma
    rdd = spark.sparkContext.parallelize(data1)
    df = spark.createDataFrame(rdd, schema)

    # DataFrame'i gösterme
    df.show(truncate=False)

    # Metni boşluklara göre ayırın ve kelimeleri bir DataFrame olarak oluşturun
    df_split = df.select(df.url, explode(split(df.data, "\\s+")).alias("kelime"))

    # Sonucu gösterin
    df_split.show(truncate=True)

    # Geçici tablo oluşturma
    df_split.createOrReplaceTempView("kelimeler_tablosu")

    # Spark SQL kullanarak tabloyu sorgulama
    spark.sql("SELECT * FROM kelimeler_tablosu").show(truncate=True)

    # CSV olarak kaydetme
    df_split.write.mode("overwrite").csv("file:///C:/Users/Mehmet/Desktop/dataset/kelimeler_tablosu", header=True)

    # Parquet olarak kaydetme
    # Parquet olarak kaydetme
    # Parquet olarak kaydetme
 #   df_split.write.parquet("output/kelimeler_tablosu.parquet")

except ServerSelectionTimeoutError as err:
    print("MongoDB bağlantı hatası:", err)



https://www.cnnturk.com/video/turkiye/ankara-tahran-hattinda-dakika-dakika-neler-yasandi-2116090


Veri başarıyla liste olarak güncellendi
#########
https://www.cnnturk.com/turkiye/galeri/baba-ve-oglunun-aci-kaderi-84-yil-once-yasamini-yitiren-baba-berzeg-ile-ilgili-yeni-2116075


Veri başarıyla liste olarak güncellendi
#########
+------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------